# Neural Network Fitting and Benchmarking

## Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
from m3util.viz.printing import printer
from m3util.ml.optimizers.TrustRegion import TRCG
from m3util.viz.style import set_style
from m3util.ml.rand import set_seeds

# JGoddy had to add these to the path because 
# I am using their local instances and they are 
# in different folders 
sys.path.append(os.path.abspath("/home/jg3837/Band_Excitation/BEPiezo-Learn/src"))
sys.path.append(os.path.abspath("/home/jg3837/DataFed_TorchFlow/DataFed_TorchFlow/src"))

from belearn.dataset.dataset import BE_Dataset
from belearn.functions.sho import SHO_nn
from belearn.nn.nn import BatchTrainer
from belearn.nn.inference import BEInference 


from datafed_torchflow.datafed import DataFed
from datafed_torchflow.pytorch import TorchViewer
from datetime import datetime

from autophyslearn.postprocessing.complex import ComplexPostProcessor
from autophyslearn.spectroscopic.nn import Multiscale1DFitter, Model


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Specify the filename and the path to save the file
filename = "/home/jg3837/DataFed_TorchFlow/DataFed_TorchFlow/SHO_example/data_raw.h5"
save_path = "./Data"


optimizer_TR = {"name": "TRCG", "optimizer": TRCG, "radius": 5, "device": "cuda", "ADAM_epochs": 2}
optimizers = ['Adam', optimizer_TR]
noise_list =   [0, 1, 2, 3, 4, 5, 6, 7, 8]
batch_size = [500, 1000, 5000, 10000]
epochs = [5]
seed =  [41, 43, 44, 45, 46]
early_stopping_time = 60*3
basepath_postfix = 'nn_benchmarks_noise'

# Original filename
csv_name = 'nn_benchmarks_noise.csv'

printing = printer(basepath='./Figures/')

set_style("printing")
set_seeds(seed=42)

# data_path = save_path + "/" + filename
data_path = filename

printing set for seaborn
Pytorch seed was set to 42
Numpy seed was set to 42


2024-11-06 16:38:33.438373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 16:38:33.451610: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 16:38:33.455546: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 16:38:33.466090: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 16:38:34.497144: W tensorflow/compiler/tf2

tensorflow seed was set to 42


In [3]:
# instantiate the dataset object
dataset = BE_Dataset(data_path, SHO_fit_func_LSQF=SHO_nn,
            datafed = "2024_SHO_Fitting/Training_Benchmarks_NN_SHO_10_21_2024", dataset_id="d/525627216")

# print the contents of the file
dataset.print_be_tree()

Unable to connect to pypi: <Fault -32500: 'RuntimeError: PyPI no longer supports the XMLRPC package_releases method. Use JSON or Simple API instead. See https://warehouse.pypa.io/api-reference/xml-rpc.html#deprecated-methods for more information.'>
No spectroscopic datasets found as attributes of /Measurement_000/Channel_000/Position_Indices
No position datasets found as attributes of /Measurement_000/Raw_Data-SHO_Fit_000/Spectroscopic_Values
/
├ Measurement_000
  ---------------
  ├ Channel_000
    -----------
    ├ Bin_FFT
    ├ Bin_Frequencies
    ├ Bin_Indices
    ├ Bin_Step
    ├ Bin_Wfm_Type
    ├ Excitation_Waveform
    ├ Noise_Floor
    ├ Noisy_Data_1
    ├ Noisy_Data_2
    ├ Noisy_Data_3
    ├ Noisy_Data_4
    ├ Noisy_Data_5
    ├ Noisy_Data_6
    ├ Noisy_Data_7
    ├ Noisy_Data_8
    ├ Position_Indices
    ├ Position_Values
    ├ Raw_Data
    ├ Spatially_Averaged_Plot_Group_000
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_S

In [4]:
batch_training = True
datafed_path = "2024_SHO_Fitting/Training_Benchmarks_NN_SHO_11_04_2024"

# Get the current date and time
current_datetime = datetime.now()

# Format the date and time in a 'pretty' format (e.g., YYYY-MM-DD_HH-MM-SS)
formatted_datetime = current_datetime.strftime('%Y-%m-%d_%H-%M-%S')

basepath = f'{formatted_datetime}_{basepath_postfix}'

trainer = BatchTrainer(
    dataset=dataset,
    optimizers=optimizers,
    noise_list=noise_list,
    batch_size=batch_size,
    epochs=epochs,
    seed=seed,
    basepath=basepath,
    datafed_path=datafed_path,
    script_path=f"{os.getcwd()}/5_nn_fitting_all.ipynb",
    early_stopping_loss=None,
    early_stopping_count=None,
    early_stopping_time=early_stopping_time,
    write_CSV="Batch_Trainging_SpeedTest.csv",
)

if batch_training == True:
    trainer.run_training(dataset) 

Pytorch seed was set to 41
Numpy seed was set to 41
tensorflow seed was set to 41
Working on combination: SHO_Adam_noise_0_batch_size_500_seed_41
Using GPU NVIDIA RTX A6000
Saving to Trained Models/SHO Fitter/2024-11-04_22-44-15_nn_benchmarks_noise/
Pytorch seed was set to 41
Numpy seed was set to 41
tensorflow seed was set to 41
Unable to connect to pypi: <Fault -32500: 'RuntimeError: PyPI no longer supports the XMLRPC package_releases method. Use JSON or Simple API instead. See https://warehouse.pypa.io/api-reference/xml-rpc.html#deprecated-methods for more information.'>


/home/jg3837/anaconda3/envs/TransformerVAE6/lib/python3.10/site-packages/torch/autograd/graph.py:768: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at /opt/conda/conda-bld/pytorch_1720538438429/work/torch/csrc/autograd/engine.cpp:1203.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Adam
epoch : 1/5, recon loss = 0.06053583
--- 17.558022499084473 seconds ---
Epoch 1, Learning Rate: 0.001
Adam
epoch : 2/5, recon loss = 0.03454326
--- 14.87330150604248 seconds ---
Epoch 2, Learning Rate: 0.001
Adam
epoch : 3/5, recon loss = 0.03416985
--- 15.652843475341797 seconds ---
Epoch 3, Learning Rate: 0.001
Adam
epoch : 4/5, recon loss = 0.03406213
--- 15.011247873306274 seconds ---
Epoch 4, Learning Rate: 0.001
Adam
epoch : 5/5, recon loss = 0.03396321
--- 15.460019588470459 seconds ---
Epoch 5, Learning Rate: 0.001
Pytorch seed was set to 43
Numpy seed was set to 43
tensorflow seed was set to 43
Working on combination: SHO_Adam_noise_0_batch_size_500_seed_43
Using GPU NVIDIA RTX A6000
Saving to Trained Models/SHO Fitter/2024-11-04_22-44-15_nn_benchmarks_noise/
Pytorch seed was set to 43
Numpy seed was set to 43
tensorflow seed was set to 43
Unable to connect to pypi: <Fault -32500: 'RuntimeError: PyPI no longer supports the XMLRPC package_releases method. Use JSON or Simpl

In [6]:
dataset.datafed_obj.replace_missing_records(collection_id='c/525610729', record_id='d/525630823',
                                            file_path ='/home/jg3837/DataFed_TorchFlow/DataFed_TorchFlow/SHO_example/5_nn_fitting_all.ipynb',
                                            )

checking collection c/525610729 for missing records


100%|██████████| 541/541 [01:57<00:00,  4.61it/s]


found 14 missing records
retrieving metadata for missing records


100%|██████████| 14/14 [00:02<00:00,  4.93it/s]


trying to reupload Early_Stoppage_at_322.30901050567627_SHO_Fitter_model_optimizer_Trust Region CG_epoch_3_train_loss_5.976587369244684.pth for record d/525631385
